In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [6]:
class TeamScraper:
    def __init__(self, url):
        if(url):
            self.response = requests.get(url)
            self.team_soup = BeautifulSoup(self.response.text, parser='lxml')

            self.stat_table = StatTableHandler(self.team_soup)
            self.team_info = TeamInfoHandler(self.team_soup)
        else:
            print("URL is None")
    def parse_team_table(self):
        print("bleh")
        
        
           
        
        #iterate through rows and get 
        
    def get_stat_csv(self):
        # Update to include team info in csv title
        self.stat_table.table_df.to_csv(self.team_info.get_name() + '.csv', index=False)
        
        
        
    def print_url(self):
        print(self.url)
        
        
class StatTableHandler:
    def __init__(self, team_soup):
        table = team_soup.find('div', {'class':'table-responsive'})
        self.table = table
        self.table_headers = self.get_table_headers()
        self.table_contents = self.get_table_contents()
        self.table_df = self.generate_dataframe()
    def get_table_headers(self):
        # Get table header for csv headers
        table_headers = self.table.find_all('th')        
        final_headers = []
        for table_header in table_headers:            
            # If the tag has attributes (ie a full name), get them
            header_attrs = table_header.attrs
            if(header_attrs):
                final_headers.append(table_header['title'])
            else:
                final_headers.append(table_header.contents[0])
        return final_headers
    def get_table_contents(self):
        table_body = self.table.find('tbody')
        rows = table_body.find_all('tr')
        final_rows = []
        # Iterate through and process table rows
        for row in rows:
            # iterate through each row's columns
            scraped_row = [td.text for td in row.find_all('td')]
            final_rows.append(scraped_row)
        return final_rows
    
    def generate_dataframe(self):
        table_df = pd.DataFrame(self.table_contents, columns = self.table_headers)
        return table_df
    
class TeamInfoHandler:
    def __init__(self, team_soup):
        name_record = team_soup.find('h1', {'class':'m-b-0'}).text.split('(')
        self.name = name_record[0].strip()
        self.record = name_record[1][:-1].strip()
    def get_name(self):
        return self.name
    def get_record(self):
        return self.record


In [7]:
# Class to handle a division
class DivisionScraper:
    def __init__(self, url):
        self.url = url
        resp = requests.get(self.url)
        division_soup = BeautifulSoup(resp.text, parser='lxml')
        table = division_soup.find('div', {'class':'box clearfix'})
        division_and_conference = table.find_all('table', {'class':'table table-striped box'})
        self.division_table = division_and_conference[0]
        self.conference_tables = division_and_conference[1:]
        self.division_links = self.get_division_links()
        
    def get_division_links(self, master_url = 'https://www.insidelacrosse.com/'):
        body = self.division_table.find('tbody')
        teams = body.find_all('tr')
        division_links = []
        for row in teams:
            #Link is the first element in the row
            link = row.find_all('td')[0].find('a')['href']
            # add stats to url
            split_link = link.split('/')
            split_link[0] = split_link[1]
            split_link[1] = 'stats'
            
            # Rejion
            final_link = '/'.join(split_link)
            division_links.append(master_url + final_link)
        return division_links
            
    
    def get_url(self):
        return self.url
    
    def get_division_table(self):
        return self.division_table
    
    # ToDo - complete this method
    def get_conference_tables(self):
        return self.conference_tables

In [8]:
div_url = 'https://www.insidelacrosse.com/league/di/teams/2021'
div_scraper = DivisionScraper(div_url)

d1_links = div_scraper.division_links

for link in d1_links:
    ts = TeamScraper(link)
    ts.get_stat_csv()

IndexError: list index out of range